In [15]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("YourAppName").getOrCreate()

from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [3]:
# Re-create iviris_df
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer

iris_df =spark.read.csv("Exercise_Files/iris.txt", inferSchema=True)

iris_df = iris_df.select(
    col("_c0").alias("sepal_length"),
    col("_c1").alias("sepal_width"),
    col("_c2").alias("petal_length"),
    col("_c3").alias("petal_width"),
    col("_c4").alias("species")
    )

vectorAssembler = VectorAssembler(inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"], outputCol="features")
viris_df = vectorAssembler.transform(iris_df)
indexer = StringIndexer(inputCol="species", outputCol="label")
iviris_df = indexer.fit(viris_df).transform(viris_df)

In [5]:
iviris_df.take(1)

[Row(sepal_length=5.1, sepal_width=3.5, petal_length=1.4, petal_width=0.2, species='Iris-setosa', features=DenseVector([5.1, 3.5, 1.4, 0.2]), label=0.0)]

In [9]:
splits = iviris_df.randomSplit([0.6, 0.4], 1)  # Split to two groups: 60% and 40%; seed = 1
train_df = splits[0]
test_df = splits[1]

print(iviris_df.count())
print(train_df.count())
print(test_df.count())

150
98
52


In [11]:
layers = [4, 5, 5, 3]  # 1st layer: 4 neurons; First layer has the same number of nodes as inputs, which is 4 measures;
                       # Middle two layerss have 5 neurons
                       # Output layer (last layer) has 3 neurons: we have 3 types of outputs

In [12]:
mlp = MultilayerPerceptronClassifier(layers=layers, seed=1)

In [13]:
mlp_model = mlp.fit(train_df)

24/02/11 23:40:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/02/11 23:40:24 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [14]:
mlp_predictions = mlp_model.transform(test_df)

In [16]:
mlp_evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

In [17]:
mlp_accuracy = mlp_evaluator.evaluate(mlp_predictions)

In [19]:
mlp_accuracy

0.6923076923076923